In [1]:
import pandas as pd 
import numpy as np
import os
import glob
from datetime import timedelta
from sklearn.preprocessing import StandardScaler

In [2]:
file_path = '/Users/antonebalakov/Downloads/train_data_Sila/data_example/data_Q2_2023/'
#считываем данные
files = ['2023-04-01.csv', '2023-04-02.csv', '2023-04-03.csv', '2023-04-04.csv', '2023-04-05.csv',
         '2023-04-06.csv', '2023-04-07.csv', '2023-04-08.csv', '2023-04-09.csv', '2023-04-10.csv']
#files = ['2023-04-01.csv', '2023-04-02.csv', '2023-04-03.csv']
data = pd.DataFrame()
i = 0
for filename in files:
    temp = pd.read_csv(file_path+filename)
    temp = temp.drop_duplicates(subset=['date', 'serial_number'])
    data = pd.concat([data, temp])
data

,date,serial_number,model,capacity_bytes,failure,vault_id,pod_id,is_legacy_format,smart_1_normalized,smart_1_raw,...,smart_250_normalized,smart_250_raw,smart_251_normalized,smart_251_raw,smart_252_normalized,smart_252_raw,smart_254_normalized,smart_254_raw,smart_255_normalized,smart_255_raw
0,2023-04-01,PL1331LAGDJ5GH,HGST HDS5C4040ALE630,4000787030016,0,1000,0,False,100.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-04-01,PL2331LAGLW5UJ,HGST HMS5C4040BLE640,4000787030016,0,1000,0,False,100.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-04-01,PL2331LAGM410J,HGST HMS5C4040BLE640,4000787030016,0,1000,0,False,100.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-04-01,PL2331LAGM5NTJ,HGST HMS5C4040BLE640,4000787030016,0,1000,0,False,100.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-04-01,PL2331LAGM5R4J,HGST HMS5C4040BLE640,4000787030016,0,1000,0,False,100.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240737,2023-04-10,3FHZ700T,WDC WUH721816ALE6L4,16000900661248,0,2016,19,False,100.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
240738,2023-04-10,3FJREXST,WDC WUH721816ALE6L4,16000900661248,0,2016,19,False,100.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
240739,2023-04-10,3FJS8G3T,WDC WUH721816ALE6L4,16000900661248,0,2016,19,False,100.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
240740,2023-04-10,3RJ4J8ZA,WDC WUH721816ALE6L4,16000900661248,0,2016,19,False,100.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
#убираем фичи где слишком много пропусков
data['date'] = pd.to_datetime(data['date'])

feature_list = []

for column in data.columns:
    if (data[column].nunique() >= 1) & (data[column].notnull().sum() > 0.05 * data.shape[0]):
        feature_list.append(column)
        
data = data[feature_list]
#считываем данные таргетов, которые произойдут через 30 дней
file_path = '/Users/antonebalakov/Downloads/train_data_Sila/data_example/data_Q3_2023/'
dates_to_failure =  ['2023-07-01.csv', '2023-07-02.csv', '2023-07-03.csv', '2023-07-04.csv', '2023-07-05.csv',
         '2023-07-06.csv', '2023-07-07.csv', '2023-07-08.csv', '2023-07-09.csv', '2023-07-10.csv']
#dates_to_failure =  ['2023-07-01.csv', '2023-07-02.csv', '2023-07-03.csv']


failure_data = pd.DataFrame()
for filename in dates_to_failure:
    temp = pd.read_csv(file_path+filename)
    temp = temp.drop_duplicates(subset=['date', 'serial_number'])[['date', 'serial_number', 'failure']]
    
    failure_data = pd.concat([failure_data, temp])
failure_data

,date,serial_number,failure
0,2023-07-01,PL1331LAGDJ5GH,0
1,2023-07-01,PL2331LAGLW5UJ,0
2,2023-07-01,PL2331LAGM410J,0
3,2023-07-01,PL2331LAGM5NTJ,0
4,2023-07-01,PL2331LAGM5R4J,0
...,...,...,...
245753,2023-07-10,ZL2L8DBB,0
245754,2023-07-10,ZL2L8DDB,0
245755,2023-07-10,ZL2L8DHC,0
245756,2023-07-10,ZL2L8VAC,0


In [4]:
#мержим данные таргетов с основными данными 
failure_data['date'] = pd.to_datetime(failure_data['date'])
failure_data['date'] = failure_data['date'] - timedelta(days=90)
failure_data = failure_data.rename(columns={'failure': 'target'})
data = data.merge(failure_data, on=['date', 'serial_number'], how='left')
failure_data

,date,serial_number,target
0,2023-04-02,PL1331LAGDJ5GH,0
1,2023-04-02,PL2331LAGLW5UJ,0
2,2023-04-02,PL2331LAGM410J,0
3,2023-04-02,PL2331LAGM5NTJ,0
4,2023-04-02,PL2331LAGM5R4J,0
...,...,...,...
245753,2023-04-11,ZL2L8DBB,0
245754,2023-04-11,ZL2L8DDB,0
245755,2023-04-11,ZL2L8DHC,0
245756,2023-04-11,ZL2L8VAC,0


In [5]:
#сортируй датафрейм (сортирутеся 5-10 минут)
data = data.sort_values(['serial_number', 'date'])

In [ ]:
#оставляем только нормализованные фичи и строим по ним EMA (экспоненциальное скользящее среднее)
smart_features = []
base_features = []
for column in data.columns:
    if column[-10:] != 'normalized' and column[:5] == 'smart':
        smart_features.append(column)
    elif column[-10:] != 'normalized':
        base_features.append(column)


data = data[base_features + smart_features]
scaler = StandardScaler()
data[smart_features] = scaler.fit_transform(data[smart_features])


#Вычисление скользящего среднего
window_size = 21
for feature in smart_features:
    data[feature+'_ema'] = data.groupby('serial_number')[feature].transform(lambda x: x.ewm(span=window_size, adjust=False).mean())
   
data

In [ ]:
#Вычисление изменений фич за 1 день, 2 дня ... 7 дней (регулируется)

size = 7
new_columns = []
for i in range(size, size+1):
    print(i)
    for feature in smart_features:
        new_col_name = f'{feature}_chg_{i}'
        new_col_data = (data[feature] /  data.groupby('serial_number')[feature].shift(i)) - 1
        new_columns.append(pd.Series(new_col_data, name=new_col_name))
        
        new_col_name = f'{feature}_ema_chg_{i}'
        new_col_data = (data[feature+'_ema'] /  data.groupby('serial_number')[feature+'_ema'].shift(i)) - 1
        new_columns.append(pd.Series(new_col_data, name=new_col_name))
        
data = pd.concat([data] + new_columns, axis=1)
data.to_csv('temp_data.csv', index=False)
data


In [ ]:
def count_binary_1(row):
    return data[(data['date'] == row['date']) &
              (data['cluster_id'] == row['cluster_id']) &
              (data['failure'] == 1)].shape[0]

# Применяем функцию к каждой строке DataFrame
data = pd.read_csv("temp_data.csv")
data['count_failure_in_group'] = data.apply(count_binary_1, axis=1)

In [ ]:
data.to_csv("data_to_train_90.csv")

In [3]:
df = pd.read_csv("data_to_train_30.csv")
df

,Unnamed: 0,date,serial_number,model,capacity_bytes,failure,datacenter,cluster_id,vault_id,pod_id,...,smart_199_normalized_chg,smart_200_normalized_chg,smart_220_normalized_chg,smart_222_normalized_chg,smart_223_normalized_chg,smart_224_normalized_chg,smart_226_normalized_chg,smart_240_normalized_chg,smart_241_normalized_chg,smart_242_normalized_chg
0,220768,2023-10-01,000a43e7dee60010,DELLBOSS VD,480036847616,0,ams5,31,2017,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,485951,2023-10-02,000a43e7dee60010,DELLBOSS VD,480036847616,0,ams5,31,2017,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,751124,2023-10-03,000a43e7dee60010,DELLBOSS VD,480036847616,0,ams5,31,2017,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1017496,2023-10-04,000a43e7dee60010,DELLBOSS VD,480036847616,0,ams5,31,2017,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1283931,2023-10-05,000a43e7dee60010,DELLBOSS VD,480036847616,0,ams5,31,2017,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5343123,4225463,2023-10-16,ffe3e8fdc9300010,DELLBOSS VD,480036847616,0,ams5,31,2018,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5343124,4493660,2023-10-17,ffe3e8fdc9300010,DELLBOSS VD,480036847616,0,ams5,31,2018,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5343125,4761853,2023-10-18,ffe3e8fdc9300010,DELLBOSS VD,480036847616,0,ams5,31,2018,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5343126,5030050,2023-10-19,ffe3e8fdc9300010,DELLBOSS VD,480036847616,0,ams5,31,2018,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
list(d.dtypes)

[dtype('int64'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('int64'),
 dtype('int64'),
 dtype('O'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('float64'),
 dtype('bool'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float

In [7]:
pd.set_option('display.max_columns', None)


In [8]:
df.iloc()[:100000].describe()

,Unnamed: 0,capacity_bytes,failure,cluster_id,vault_id,pod_id,pod_slot_num,target,smart_1_normalized,smart_2_normalized,smart_3_normalized,smart_4_normalized,smart_5_normalized,smart_7_normalized,smart_8_normalized,smart_9_normalized,smart_10_normalized,smart_12_normalized,smart_18_normalized,smart_22_normalized,smart_23_normalized,smart_24_normalized,smart_173_normalized,smart_174_normalized,smart_183_normalized,smart_184_normalized,smart_187_normalized,smart_188_normalized,smart_189_normalized,smart_190_normalized,smart_191_normalized,smart_192_normalized,smart_193_normalized,smart_194_normalized,smart_195_normalized,smart_196_normalized,smart_197_normalized,smart_198_normalized,smart_199_normalized,smart_200_normalized,smart_220_normalized,smart_222_normalized,smart_223_normalized,smart_224_normalized,smart_226_normalized,smart_240_normalized,smart_241_normalized,smart_242_normalized,smart_1_normalized_ema,smart_2_normalized_ema,smart_3_normalized_ema,smart_4_normalized_ema,smart_5_normalized_ema,smart_7_normalized_ema,smart_8_normalized_ema,smart_9_normalized_ema,smart_10_normalized_ema,smart_12_normalized_ema,smart_18_normalized_ema,smart_22_normalized_ema,smart_23_normalized_ema,smart_24_normalized_ema,smart_173_normalized_ema,smart_174_normalized_ema,smart_183_normalized_ema,smart_184_normalized_ema,smart_187_normalized_ema,smart_188_normalized_ema,smart_189_normalized_ema,smart_190_normalized_ema,smart_191_normalized_ema,smart_192_normalized_ema,smart_193_normalized_ema,smart_194_normalized_ema,smart_195_normalized_ema,smart_196_normalized_ema,smart_197_normalized_ema,smart_198_normalized_ema,smart_199_normalized_ema,smart_200_normalized_ema,smart_220_normalized_ema,smart_222_normalized_ema,smart_223_normalized_ema,smart_224_normalized_ema,smart_226_normalized_ema,smart_240_normalized_ema,smart_241_normalized_ema,smart_242_normalized_ema,smart_1_normalized_chg,smart_2_normalized_chg,smart_3_normalized_chg,smart_4_normalized_chg,smart_5_normalized_chg,smart_7_normalized_chg,smart_8_normalized_chg,smart_9_normalized_chg,smart_10_normalized_chg,smart_12_normalized_chg,smart_18_normalized_chg,smart_22_normalized_chg,smart_23_normalized_chg,smart_24_normalized_chg,smart_173_normalized_chg,smart_174_normalized_chg,smart_183_normalized_chg,smart_184_normalized_chg,smart_187_normalized_chg,smart_188_normalized_chg,smart_189_normalized_chg,smart_190_normalized_chg,smart_191_normalized_chg,smart_192_normalized_chg,smart_193_normalized_chg,smart_194_normalized_chg,smart_195_normalized_chg,smart_196_normalized_chg,smart_197_normalized_chg,smart_198_normalized_chg,smart_199_normalized_chg,smart_200_normalized_chg,smart_220_normalized_chg,smart_222_normalized_chg,smart_223_normalized_chg,smart_224_normalized_chg,smart_226_normalized_chg,smart_240_normalized_chg,smart_241_normalized_chg,smart_242_normalized_chg
count,1.000000e+05,1.000000e+05,100000.000000,100000.000000,100000.000000,100000.000000,99392.000000,99897.000000,99393.0,99393.000000,99393.0,99393.0,99393.000000,99393.0,99393.0,99393.000000,99393.0,99393.0,0.0,0.0,99393.0,99393.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,90841.0,99393.0,99393.0,99393.0,0.0,99393.0,90841.0,99393.000000,99393.0,0.0,90841.0,90841.000000,90841.0,90841.0,90841.0,90841.0,8552.0,8552.0,99393.0,99393.000000,99393.0,99393.0,99393.000000,99393.0,99393.0,99393.000000,99393.0,99393.0,0.0,0.0,99393.0,99393.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,90841.0,99393.0,99393.0,99393.0,0.0,99393.0,90841.0,99393.000000,99393.0,0.0,90841.0,90841.000000,90841.0,90841.0,90841.0,90841.0,8552.0,8552.0,64570.0,64570.000000,64570.0,64570.0,64570.0,64570.0,64570.0,64570.000000,64570.0,64570.0,0.0,0.0,64570.0,6.457000e+04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,59028.0,64570.0,64570.0,64570.0,0.0,64570.0,59028.0,64570.0,64570.0,0.0,59028.0,59028.000000,59028.0,59028.0,59028.0,59028.0,5542.0,5542.0
mean,2.639159e+06,1.395806e+13,0.000050,0.801970,1122.432440,9.210580,28.060548,0.000020,100.0,99.972835,100.0,100.0,99.997183,100.0,100.0,3

In [14]:
base_features

['date',
 'serial_number',
 'model',
 'capacity_bytes',
 'failure',
 'vault_id',
 'pod_id',
 'is_legacy_format',
 'target']